# LeNet-5 실습 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

### CIFAR-10 데이터 셋 로드

In [2]:
# transfrom
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # 이미지를 정규화
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # 이미지를 정규화
])

In [3]:


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)
testloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=False)



Files already downloaded and verified
Files already downloaded and verified


### LeNet-5 모델

In [7]:
import torch
import torch.nn as nn

class RFNet(nn.Module):
    def __init__(self):
        super(RFNet, self).__init__()
        # Convolutional layer 1: 입력 채널 수 3, 출력 채널 수 32, 커널 크기 3x3
        self.conv1 = nn.Conv2d(3, 32, 3)
        # Max Pooling layer: 풀링 영역 크기 2x2, 스트라이드 2x2
        self.pool = nn.MaxPool2d(2, 2)
        # Convolutional layer 2: 입력 채널 수 32, 출력 채널 수 64, 커널 크기 3x3
        self.conv2 = nn.Conv2d(32, 64, 3)
        # Fully Connected (Linear) layer 1: 입력 노드 수 64 * 6 * 6, 출력 노드 수 64
        self.fc1 = nn.Linear(64 * 6 * 6, 64)
        # Fully Connected (Linear) layer 2: 입력 노드 수 64, 출력 노드 수 10
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        # 입력 이미지를 Convolutional layer 1에 통과시키고 ReLU 활성화 함수를 적용하여 feature map 생성
        x = torch.relu(self.conv1(x))  # feature map 크기: 14 x 14 (입력 크기 - 커널 크기 + 1)
        # Max Pooling을 적용하여 feature map 크기를 반으로 줄임
        x = self.pool(x)  # feature map 크기: 7 x 7 (입력 크기 // 2)
        
        # 입력 feature map을 Convolutional layer 2에 통과시키고 ReLU 활성화 함수를 적용하여 feature map 생성
        x = torch.relu(self.conv2(x))  # feature map 크기: 5 x 5 (입력 크기 - 커널 크기 + 1)
        # Max Pooling을 적용하여 feature map 크기를 반으로 줄임
        x = self.pool(x)  # feature map 크기: 2 x 2 (입력 크기 // 2)

        # 2차원 feature map을 1차원으로 변환 (Flatten)
        x = torch.flatten(x, 1)  # 64개의 2차원 feature map을 1차원으로 펼쳐서 크기가 64 * 2 * 2인 벡터 생성
        
        # Fully Connected layer 1에 통과시키고 ReLU 활성화 함수를 적용하여 출력 생성
        x = torch.relu(self.fc1(x))

        # Fully Connected layer 2에 통과시키고 최종 출력 생성
        x = self.fc2(x)

        return x


In [19]:
#  모델 학습 및 평가 함수
def train_and_evaluate(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    # 학습 loop
    for epoch in range(5):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            
            '''
            매 200번째 미니 배치마다 조건이 참이 되어 손실이 출력되고 running_loss가 재설정됩니다.
            이는 미니 배치가 190번째일 때마다 출력 및 초기화가 수행됩니다.
            '''
            if i % 200 == 199:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
                running_loss = 0.0
                
    print('Finished training')
    
    # 모델 평가
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = 100 * correct / total
    print('Acc >>  %.2f%%' % accuracy)

### 실행

In [20]:
print("RNFet-5")
rf_model = RFNet()
train_and_evaluate(rf_model)

# 파라미터 수 비교
print("Number of parameters:")
print("RFNet: ", sum(p.numel() for p in rf_model.parameters()))

RNFet-5
[1,   200] loss: 2.293
[1,   400] loss: 2.243
[1,   600] loss: 2.088
[2,   200] loss: 1.888
[2,   400] loss: 1.839
[2,   600] loss: 1.792
[3,   200] loss: 1.671
[3,   400] loss: 1.621
[3,   600] loss: 1.571
[4,   200] loss: 1.503
[4,   400] loss: 1.488
[4,   600] loss: 1.444
[5,   200] loss: 1.396
[5,   400] loss: 1.400
[5,   600] loss: 1.370
Finished training
Acc >>  52.57%
Number of parameters:
RFNet:  167562
